In [5]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install shortuuid

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install aiofile

Note: you may need to restart the kernel to use updated packages.


In [10]:
import psycopg, os
from PIL import Image
from IPython.display import display
import pandas as pd
from tabulate import tabulate
from pymongo import MongoClient
import bcrypt
import shortuuid
import json
import os
import asyncio
from aiofile import AIOFile, LineReader
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

In [11]:

print('Connecting to the PostgreSQL database...')
try:
    conn = psycopg.connect(
        host="apan-postgres",
        port='5432',
        dbname="db",
        user="admin",
        password="PassW0rd")

    # create a cursor
    cur = conn.cursor()

    print('Database connected successfully.\nPostgreSQL database version:')
    cur.execute('SELECT version()')
    db_version = cur.fetchone()
    print(db_version)
except Exception as e:
    print(f"Error connecting to the database:{e}")

Connecting to the PostgreSQL database...
Database connected successfully.
PostgreSQL database version:
('PostgreSQL 17.3 (Debian 17.3-3.pgdg120+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [2]:
import base64
import uuid

def uuid_encode(uuid_str):
    uuid_bytes = uuid.UUID(uuid_str).bytes
    encoded_uuid = base64.urlsafe_b64encode(uuid_bytes).rstrip(b'=').decode('utf-8')
    return encoded_uuid

def uuid_decode(encoded_uuid):
    padded_encoded_uuid = encoded_uuid + '=='  # Add padding back
    uuid_bytes = base64.urlsafe_b64decode(padded_encoded_uuid)
    decoded_uuid = str(uuid.UUID(bytes=uuid_bytes))
    return decoded_uuid

In [4]:
uuid_decode('qVc8ODYU5SZjKXVBgXdI7w')

'a9573c38-3614-e526-6329-7541817748ef'

In [5]:
uuid_encode('a9573c38-3614-e526-6329-7541817748ef')

'qVc8ODYU5SZjKXVBgXdI7w'

In [14]:

# file_path = os.path.join(os.path.dirname(__file__), 'yelp_academic_dataset_user_100.json')
file_path = os.path.join('yelp_academic_dataset_user_100.json')

chunk_size = 20
last_line = 0

async def read_lines(file_path, start_line, chunk_size):
    lines = []
    async with AIOFile(file_path, 'r') as afp:
        reader = LineReader(afp)
        async for line_number, line in enumerate(reader):
            if line_number >= start_line and line_number < start_line + chunk_size:
                lines.append(json.loads(line))
            if line_number >= start_line + chunk_size:
                break
    return lines

async def up(query_interface, sequelize):
    transaction = query_interface.begin()
    translator = shortuuid.ShortUUID(alphabet=shortuuid.get_alphabet('cookieBase90'))

    try:
        data = await read_lines(file_path, last_line, chunk_size)

        users_chunk = [{
            'uuid': translator.uuid(user['user_id']),
            'first_name': user['name'],
            'last_name': user.get('last_name', ''),
            'email': '',
            'phone': '',
            'password': bcrypt.hashpw(user['password'].encode('utf-8'), bcrypt.gensalt()).decode('utf-8'),
            'createdAt': datetime.now(),
            'updatedAt': datetime.now()
        } for user in data]

        users_table = Table('users_user', MetaData(), autoload_with=query_interface)
        query_interface.execute(users_table.insert(), users_chunk)
        last_line += chunk_size

        transaction.commit()
    except Exception as e:
        transaction.rollback()
        with open(os.path.join('lastLine.txt'), 'w') as f:
            f.write(str(last_line))
        raise e

async def down(query_interface, sequelize):
    try:
        users_table = Table('users_user', MetaData(), autoload_with=query_interface)
        query_interface.execute(users_table.delete())
    except Exception as e:
        raise e
